In [1]:
import warnings
warnings.filterwarnings('ignore')

## 1. KNN (K-Nearest Neighbor)

In [2]:
! wget ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz
! tar -xf sift.tar.gz
! mkdir data/sift1M -p
! mv sift/* data/sift1M

--2025-02-28 20:28:26--  ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz
           => ‘sift.tar.gz’
Resolving ftp.irisa.fr (ftp.irisa.fr)... 131.254.254.45, 2001:660:7303:254::45
Connecting to ftp.irisa.fr (ftp.irisa.fr)|131.254.254.45|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /local/texmex/corpus ... done.
==> SIZE sift.tar.gz ... 168280445
==> PASV ... done.    ==> RETR sift.tar.gz ... done.
Length: 168280445 (160M) (unauthoritative)

sift.tar.gz         100%[===================>] 160.48M  10.7MB/s    in 14s     

2025-02-28 20:28:41 (11.8 MB/s) - ‘sift.tar.gz’ saved [168280445]



In [3]:
import psutil

def get_memory_usage_mb():
  process = psutil.Process()
  memory_info = process.memory_info()
  return memory_info.rss / (1024 * 1024)

In [4]:
%%capture
! pip install faiss-cpu

In [5]:
import time
import faiss
from faiss.contrib.datasets import DatasetSIFT1M

ds = DatasetSIFT1M()

xq = ds.get_queries()
xb = ds.get_database()
gt = ds.get_groundtruth()

In [6]:
k = 1
d = xq.shape[1]
nq = 1000
xq = xq[:nq]

init_memory = get_memory_usage_mb()
for i in range(1, 10, 2):
  start_memory = get_memory_usage_mb()
  start_indexing = time.time()

  index = faiss.IndexFlatL2(d)
  index.add(xb[:(i+1)*100000])

  end_indexing = time.time()
  end_memory = get_memory_usage_mb()

  t0 = time.time()
  D, I = index.search(xq, k)
  t1 = time.time()

  indexing = (end_indexing - start_indexing) * 1000
  # memory = end_memory - start_memory
  memory = end_memory - init_memory
  search = (t1 - t0) * 1000 / nq

  print(f"데이터 {(i+1)*100000}개:")
  print(f"색인: {indexing:.3f} ms ({memory:.3f} MB) 검색: {search:.3f} ms")

데이터 200000개:
색인: 103.144 ms (97.805 MB) 검색: 3.984 ms
데이터 400000개:
색인: 300.760 ms (196.738 MB) 검색: 6.104 ms
데이터 600000개:
색인: 250.059 ms (298.484 MB) 검색: 5.316 ms
데이터 800000개:
색인: 345.282 ms (396.098 MB) 검색: 4.838 ms
데이터 1000000개:
색인: 433.745 ms (493.766 MB) 검색: 7.660 ms


## 2. HNSW - parameter: m

In [7]:
import numpy as np

k = 1
d = xq.shape[1]
nq = 1000
xq = xq[:nq]

init_memory = get_memory_usage_mb()
for m in [8, 16, 32, 64]:
  index = faiss.IndexHNSWFlat(d, m)
  time.sleep(3)

  start_memory = get_memory_usage_mb()
  start_indexing = time.time()

  index.add(xb)

  end_indexing = time.time()
  end_memory = get_memory_usage_mb()

  indexing = (end_indexing - start_indexing)
  # memory = end_memory - start_memory
  memory = end_memory - init_memory

  print(f"M: {m} - 색인: {indexing} s ({memory:.3f} MB)")

  t0 = time.time()
  D, I = index.search(xq, k)
  t1 = time.time()

  per_query = (t1 - t0) * 1000.0 / nq
  recall_at_1 = np.equal(I, gt[:nq, :1]).sum() / float(nq)

  print(f"{per_query:.3f} ms/query, R@1 {recall_at_1:.3f}")

M: 8 - 색인: 122.96884989738464 s (80.742 MB)
0.046 ms/query, R@1 0.686
M: 16 - 색인: 135.1342692375183 s (141.668 MB)
0.116 ms/query, R@1 0.791
M: 32 - 색인: 251.60815334320068 s (263.562 MB)
0.104 ms/query, R@1 0.909
M: 64 - 색인: 337.6571247577667 s (507.914 MB)
0.151 ms/query, R@1 0.936


## 3. HNSW - parameter: ef_construction

In [9]:
k = 1
d = xq.shape[1]
nq = 1000
xq = xq[:nq]
m = 32

init_memory = get_memory_usage_mb()
for ef_construction in [40, 80, 160, 320]:
  index = faiss.IndexHNSWFlat(d, m)
  index.hnsw.efConstruction = ef_construction
  time.sleep(3)

  start_memory = get_memory_usage_mb()
  start_indexing = time.time()

  index.add(xb)

  end_indexing = time.time()
  end_memory = get_memory_usage_mb()

  indexing = (end_indexing - start_indexing)
  # memory = end_memory - start_memory
  memory = end_memory - init_memory

  print(f"efConstruction: {ef_construction} - 색인: {indexing} s ({memory:.3f} MB)")

  t0 = time.time()
  D, I = index.search(xq, k)
  t1 = time.time()

  per_query = (t1 - t0) * 1000.0 / nq
  recall_at_1 = np.equal(I, gt[:nq, :1]).sum() / float(nq)

  print(f"{per_query:.3f} ms/query, R@1 {recall_at_1:.3f}")

efConstruction: 40 - 색인: 236.34866738319397 s (748.172 MB)
0.108 ms/query, R@1 0.907
efConstruction: 80 - 색인: 291.94095730781555 s (748.117 MB)
0.076 ms/query, R@1 0.863
efConstruction: 160 - 색인: 586.52268409729 s (748.062 MB)
0.087 ms/query, R@1 0.906
efConstruction: 320 - 색인: 1143.8348817825317 s (748.266 MB)
0.092 ms/query, R@1 0.915


## 4. HNSW - parameter: ef_search

In [11]:
k = 1
d = xq.shape[1]
nq = 1000
xq = xq[:nq]
m = 32
ef_construction = 320

init_memory = get_memory_usage_mb()
for ef_search in [16, 32, 64, 128]:
  index = faiss.IndexHNSWFlat(d, m)
  index.hnsw.efConstruction = ef_construction
  index.hnsw.efSearch = ef_search
  time.sleep(3)

  start_memory = get_memory_usage_mb()
  start_indexing = time.time()

  index.add(xb)

  end_indexing = time.time()
  end_memory = get_memory_usage_mb()

  indexing = (end_indexing - start_indexing)
  # memory = end_memory - start_memory
  memory = end_memory - init_memory

  print(f"efSearch: {ef_search} - 색인: {indexing} s ({memory:.3f} MB)")

  t0 = time.time()
  D, I = index.search(xq, k)
  t1 = time.time()

  per_query = (t1 - t0) * 1000.0 / nq
  recall_at_1 = np.equal(I, gt[:nq, :1]).sum() / float(nq)

  print(f"{per_query:.3f} ms/query, R@1 {recall_at_1:.3f}")

efSearch: 16 - 색인: 1133.8854548931122 s (747.914 MB)
0.091 ms/query, R@1 0.918
efSearch: 32 - 색인: 1143.8217668533325 s (748.375 MB)
0.165 ms/query, R@1 0.969
efSearch: 64 - 색인: 1142.7180523872375 s (748.320 MB)
0.277 ms/query, R@1 0.985
efSearch: 128 - 색인: 1144.839806318283 s (748.523 MB)
0.468 ms/query, R@1 0.991
